In [1]:


import numpy as np
import os, platform
import winsound
import sys
import keras
import keras.backend as k
from keras.layers import Conv2D,MaxPooling2D,SpatialDropout2D,Flatten,Dropout,Dense
from keras.models import Sequential,load_model
from keras.optimizers import adam
from keras.preprocessing import image
import cv2
import datetime


Using TensorFlow backend.


In [ ]:

model=Sequential()


model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

model.summary()

from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        'D:\\DIP PROJECT\\FaceMaskDetector-master\\train',
        target_size=(150,150),
        batch_size=16 ,
    #    color_mode="grayscale",   
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        'D:\\DIP PROJECT\\FaceMaskDetector-master\\test',
        target_size=(150,150),
        batch_size=16,
    #    color_mode="grayscale",  
        class_mode='binary')

model_saved=model.fit_generator(
        generator=training_set,
        epochs=10,
        validation_data=test_set,

        )

model.save('mymodel.h5',model_saved)




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9248)             

In [ ]:
#To test for individual images

test_image=image.load_img('D:\\DIP PROJECT\\FaceMaskDetector-master\\img\\blue_mask_on_white_table.jpg',target_size=(150,150,3))

test_image
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
a=model.predict_classes(test_image)[0][0]
if (a==0):
    print('Mask Detected')
else:
    print('No Mask Detected')




In [ ]:



cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier('D:\\DIP PROJECT\\FaceMaskDetector-master\\haarcascade_frontalface_default.xml')
frequency = 2500  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second
os_name=platform.system()
while cap.isOpened():
    _,img=cap.read()
    face=face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4)
    for(x,y,w,h) in face:
        face_img = img[y:y+h, x:x+w]
        cv2.imwrite('temp.jpg',face_img)
        test_image=image.load_img('temp.jpg',target_size=(150,150,3))
        test_image=image.img_to_array(test_image)
        test_image=np.expand_dims(test_image,axis=0)
        pred=model.predict_classes(test_image)[0][0]
        
        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,'NO MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
            if (os_name=='Windows'):  #For Windows Operating System
                winsound.Beep(frequency, duration)
                
            elif (os_name=='Darwin'): #For MAC Operating System
                os.system('Say "Mask Not Detected"')
                sys.stdout.write('\a')
                sys.stdout.flush()
            cv2.putText(img,"PLEASE WEAR YOUR MASK",(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.putText(img,'MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
            cv2.putText(img,"WELCOME TO OUR SHOP",(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
       # datet=str(datetime.datetime.now())
      #  cv2.putText(img,datet,(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
          
    cv2.imshow('img',img)
    
    if cv2.waitKey(1)==ord('q'):   
        break
    
cap.release()
cv2.destroyAllWindows()
cv2.VideoCapture.release(cap)
